In [3]:

import pandas as pd
%matplotlib inline

import matplotlib

import matplotlib.pyplot as plt

import numpy as np
import sys
import os
from collections import defaultdict

import re
from nltk.tokenize import sent_tokenize,word_tokenize

import datetime

import seaborn as sns

In [5]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']

users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols,encoding='latin-1')

In [6]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols,encoding='latin-1')

In [7]:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',

'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',

'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols,

encoding='latin-1')

In [29]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_train = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')

ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')

ratings_train.shape, ratings_test.shape

((90570, 4), (9430, 4))

In [9]:
n_users = ratings.user_id.unique().shape[0]

n_items = ratings.movie_id.unique().shape[0]


In [11]:
#data matrix for users-movie items
data_matrix = np.zeros((n_users,n_items))
for line in ratings.itertuples():
    data_matrix[line[1]-1,line[2]-1]=line[3]
print(data_matrix)

[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]


In [12]:
from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(data_matrix, metric='cosine')
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')

In [18]:
def predict (ratings, similarity, type='user'):
    #base on user_simility
    if type =='user':
        mean_user_rating=ratings.mean(axis=1)
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis]+similarity.dot(ratings_diff)/np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])#求评价均值
    return pred

In [27]:
ratings_train_matrix = np.zeros((n_users, n_items))
for line in ratings_train.itertuples():
    ratings_train_matrix[line[1] - 1, line[2] - 1] = line[3]
    
print('train_data_matrix\n%s'%ratings_train_matrix)
ratings_train.shape, ratings_test.shape

train_data_matrix
[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]


((90570, 4), (9430, 4))

In [30]:
ratings_test_matrix = np.zeros((n_users, n_items))
for line in ratings_test.itertuples():
    ratings_test_matrix[line[1] - 1, line[2] - 1] = line[3]
    
print('test_data_matrix\n%s'%ratings_test_matrix)

test_data_matrix
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [34]:
item_prediction = predict(ratings_train_matrix, item_similarity, type='item')
user_prediction = predict(ratings_train_matrix, user_similarity, type='user')
print(item_prediction[0:8])
item_prediction.shape

[[0.42829293 0.45688096 0.48665064 ... 0.56533437 0.55081989 0.54486145]
 [0.09140871 0.11292361 0.10638783 ... 0.11355799 0.11558006 0.11648205]
 [0.07326828 0.07776272 0.07433974 ... 0.07202173 0.07596419 0.07691564]
 ...
 [0.34058136 0.38330375 0.40200963 ... 0.43648828 0.42848444 0.42785534]
 [0.76040268 0.78423241 0.85541801 ... 0.93217922 0.9105237  0.91028755]
 [0.0775479  0.07872801 0.09601956 ... 0.11294275 0.10818472 0.10978579]]


(943, 1682)

In [220]:
df = pd.DataFrame(item_prediction).T.apply(lambda x:x.nlargest(3))

In [421]:
df2=pd.DataFrame(item_prediction).iloc[0:10, 0: ].T.apply(lambda x:x.nlargest(3)).T
df2

,93,394,1255,1333,1358,1379,1383,1386,1390,1545,1547,1548,1553,1556,1577,1643
0,NaN,NaN,NaN,NaN,NaN,NaN,0.584841,0.587205,NaN,NaN,NaN,NaN,0.583667,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.122827,0.122827,NaN,NaN,0.122827,NaN,NaN
2,0.080234,0.080341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.080391,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.038375,0.038375,NaN,0.038663,NaN,NaN,NaN
4,NaN,NaN,0.301264,NaN,NaN,0.29941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.299784
5,NaN,NaN,NaN,0.454058,0.454058,NaN,NaN,0.454475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,0.966963,NaN,NaN,NaN,0.965329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.966170
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.117695,0.117437,NaN,0.117422,NaN
8,NaN,NaN,NaN,0.031201,NaN,NaN,NaN,0.031323,NaN,NaN,NaN,NaN,0.031339,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,0.455102,0.456342,0.453884,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [466]:
df4=pd.DataFrame(df2.T.rank()[0].dropna(axis =0, how ='any', inplace = False))
df4=pd.DataFrame(df4.sort_values(by=0,ascending=False).index).T.rename(columns={0:'1st',1:'2nd',2:'3rd'})
df4

,1st,2nd,3rd
0,1386,1383,1553


In [471]:
df5=pd.DataFrame(pd.DataFrame(df2.T.rank()[1].dropna(axis=0,how='any',inplace=False)).sort_values(by=1,ascending=False).index).T.rename(columns={0:'1st',1:'2nd',2:'3rd'})
df5.reindex(["1"])

,1st,2nd,3rd
1,NaN,NaN,NaN


In [472]:
df4=pd.concat([df4,df5],axis=0)
df4


,1st,2nd,3rd
0,1386,1383,1553
0,1545,1547,1556


In [482]:
dfx=pd.DataFrame(columns={'1st','2nd','3rd'})
for i in range(10):
    dfy=pd.DataFrame(pd.DataFrame(df2.T.rank()[i].dropna(axis=0,how='any',inplace=False)).sort_values(by=i,ascending=False).index).T.rename(columns={0:'1st',1:'2nd',2:'3rd'})
    dfx=pd.concat([dfx,dfy],axis=0,sort=True)
dfx=dfx.reset_index()
del dfx['index']
print(dfx)

    1st   2nd   3rd
0  1386  1383  1553
1  1545  1547  1556
2  1553   394    93
3  1553  1545  1547
4  1255  1643  1379
5  1386  1333  1358
6  1255  1643  1383
7  1548  1553  1577
8  1553  1386  1333
9  1386  1383  1390
